# Frozen-Lake with Q-Learning (table based)

## The environment

(From https://www.gymlibrary.ml/environments/toy_text/frozen_lake/ ):

Frozen lake involves crossing a frozen lake from Start(S) to Goal(G) without falling into any Holes(H) by walking over the Frozen(F) lake. The agent may not always move in the intended direction due to the slippery nature of the frozen lake.

### Action Space

The agent takes a 1-element vector for actions. The action space is ```(dir)```, where ```dir``` decides direction to move in which can be:

    0: LEFT

    1: DOWN

    2: RIGHT

    3: UP

### Observation Space / State Space

The observation/state is a value representing the agent’s current position as current_row * nrows + current_col (where both the row and col start at 0). For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15. The number of possible observations is dependent on the size of the map. For example, the 4x4 map has 16 possible observations.

### Rewards

Reward schedule:

    Reach goal(G): +1

    Reach hole(H): 0

    Reach frozen(F): 0


## Interacting with OpenAI gym and the environment

### Initialization

We can create and render the environment like this:

In [48]:
import gym

environment = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)

def init_environment(env: gym.Env):
    state, _ = env.reset(return_info=True)  # Restart/initialize the environment
    print(env.render(mode="ansi"))

    # The state returned from environment.reset() is our initial state:
    print(state)

init_environment(environment)


SFFF
FHFH
FFFH
HFFG

0


### Moving / Taking an action


In [49]:
# We can get the action-space with environment.action_space
print(environment.action_space)

Discrete(4)


In [50]:
# To move around (taking a series of actions), we use the ```environment.step()``` function:
def move(env: gym.Env):
    new_state, reward, done, _ = env.step(0)  # 0 is left, remember that we can use all 4 actions from the action-space, and that there is a chance of slipping

    # We got three new variables:
    print(f"New state : {new_state}")  # Updated state after moving
    print(f"Reward: {reward}")  # The reward we got from the environment
    print(f"Done: {done}")  # If the game is finished or not
    print("")
    print(env.render(mode="ansi"))  # Re-render environment after moving

move(environment)

New state : 0
Reward: 0.0
Done: False

  (Left)
SFFF
FHFH
FFFH
HFFG



## The Q-Table

We use a q-table to help guide us to the best action to take at each timestep.
It is just a simple lookup table containing the maximum expected future rewards for all actions in all states.


In [51]:
import numpy as np

# Create an empty q-table of size state-space x action-space
def initialize_q_table(env: gym.Env) -> np.array:
    return np.array([[0 for _ in range(env.action_space.n)] for _ in range(env.observation_space.n)], dtype=np.float32)

q_table = initialize_q_table(environment)
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Policies

A policy is a method that maps from a state to an action
This is what decides how we play the game

In [52]:
# We first define the optimal policy, this policy always picks what it believes is the "best" 
# action in the current state
def optimal_policy(env: gym.Env, q_sa: np.array, s: int) -> int:
    """RL-policy for optimal play.

    Args:
        env: Frozen-lake Environment
        q_sa: q-table
        s: state

    Returns:
        optimal action for given state and q-table.
    """
    if np.all(q_sa[s] == q_sa[s][0]):  # If all q-values are equal (e.g. all 0), we cannot differentiate
        return env.action_space.sample()  # Pick a random action
    return int(np.argmax(q_sa[s]))  # Return the argument (element number) with the highest q-value

In [53]:
# When we are training, we also want some exploration (see exploration/exploitation tradeoff)
def epsilon_greedy_policy(env: gym.Env, q_sa: np.array, s: int, eps: float = 0.15) -> int:
    """RL-policy for exploration/exploitation play.

    Args:
        env: Frozen-lake Environment
        q_sa: q-table
        s: state
        eps: exploration chance

    Returns:
        either random action, or optimal action for given state and q-table.
    """
    if np.random.rand() < eps:  # If a random number n is lower than eps:
        return env.action_space.sample()  # Pick a random action
    return optimal_policy(env, q_sa, s)  # Otherwise, play optimally

In [54]:
# We can also have a decaying explore strategy, to promote exploration early on and exploitation later
def decaying_epsilon_greedy_policy(env: gym.Env, q_sa: np.array, s: int, 
                                   episode: int, max_episodes: int, 
                                   max_eps: float = 0.8, min_eps: float = 0.02) -> int:
    """RL-policy for exploration/exploitation play.

    Args:
        env: Frozen-lake Environment
        q_sa: q-table
        s: state
        episode: current timestep
        max_episodes: maximum timestep
        max_eps: max exploration chance
        min_eps: min exploration chance

    Returns:
        either random action, or optimal action for given state and q-table.
    """
    eps = min_eps + (max_eps - min_eps) * ((max_episodes - episode) / max_episodes)
    if np.random.rand() < eps:  # If a random number n is lower than eps:
        return env.action_space.sample()  # Pick a random action
    return optimal_policy(env, q_sa, s)  # Otherwise, play optimally

## Playing FrozenLake with a policy

To play FrozenLake with a q-table and a policy, we just replace the action in the earlier step with the action from the policy:

In [55]:
def play():
    max_steps = 20  # Maximum steps to play
    state, _ = environment.reset(return_info=True)  # Restart/initialize the environment
    print(environment.render(mode="ansi"))
    for _ in range(max_steps):
        action = optimal_policy(environment, q_table, state)  # Chose the optimal action based on values from the q-table
        new_state, reward, done, _ = environment.step(action)  # Play using that action
        print(environment.render(mode="ansi"))

        # We stop the game if we are finished
        if done:
            break

        state = new_state  # If not, replace the state with the new state before next step

play()


SFFF
FHFH
FFFH
HFFG

  (Right)
SFFF
FHFH
FFFH
HFFG

  (Left)
SFFF
FHFH
FFFH
HFFG

  (Down)
SFFF
FHFH
FFFH
HFFG



## Learning from experience / Updating the q-table

Right now the q-table is filled with zeroes, and does not update, we want to update this for every step we take based on the td-error.

In [56]:
# The td-error is based on a single experience, i.e. a e_t = (s_t, a_t, r_t, s_(t+1)) tuple, 
# for the experience at timestep t
# We can save this in a dataclass:

from dataclasses import dataclass

@dataclass
class Experience:
    __slots__ = ("state", "action", "reward", "new_state", "done")  
    # Optimization so that we can save thousands of these objects later
    state: int
    action: int
    reward: float
    new_state: int
    done: bool

In [57]:
# We can then calculate the td-error and q-update of a single experience
def q_temporal_difference(q_sa: np.array, experience: Experience, alpha: float = 0.85, gamma: float = 0.98) -> float:
    """Calculates the q-update.

	Args:
		q_sa: q-table
		experience: a single experience
		alpha: learning-rate
		gamma: discount

	Returns:
		q-td update value
    """
    td_error = experience.reward + gamma * np.max(q_sa[experience.new_state]) - q_sa[experience.state][experience.action]
    return q_sa[experience.state][experience.action] + alpha * td_error

In [58]:
# Using the q-update, we can update our q-table over multiple games
def q_learning(env: gym.Env, q_sa: np.array, n_episodes: int = 100000, m_ep_length: int = 200) -> np.array:
    """q-learning implementation to update a q-table.

	Args:
		env: gym environment
		q_sa: initial q-table
		n_episodes: number of episodes to train on
		m_ep_length: maximum episode length

	Returns:
		updated q-table
    """
    for episode in range(n_episodes):
        s, _ = env.reset(return_info=True)  # Restart/initialize the environment
        for _ in range(m_ep_length):
            a = decaying_epsilon_greedy_policy(env, q_sa, s, episode, n_episodes)  # Exploration strategy
            s_new, r, d, _ = env.step(a)  # Play using that action

            exp = Experience(s, a, r, s_new, d)  # We create an experience from this transition
            q_td = q_temporal_difference(q_sa, exp)  # We calculate the q-update
            q_sa[s][a] = q_td  # We update the q-table

            # We stop the game if we are finished
            if d:
                break

            s = s_new  # If not, replace the state with the new state before next step
    return q_sa

In [59]:
# Now we can try:
q_table = initialize_q_table(environment)
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [73]:
q_table = q_learning(environment, q_table, n_episodes=300000)  # 100k episodes will take some time, lower to see how it works (though it might not converge)
print(np.around(q_table, 2))  # We round to two decimal places for readability

[[0.54 0.39 0.24 0.29]
 [0.   0.06 0.06 0.36]
 [0.18 0.15 0.04 0.15]
 [0.   0.01 0.01 0.15]
 [0.64 0.01 0.01 0.38]
 [0.   0.   0.   0.  ]
 [0.13 0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.12 0.1  0.02 0.59]
 [0.11 0.73 0.11 0.09]
 [0.94 0.   0.   0.06]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.63 0.7  0.83 0.11]
 [0.29 1.   0.2  0.35]
 [0.   0.   0.   0.  ]]


You can now redo the "Playing FrozenLake with a policy" step, and you have a fully working q-learning agent.